# Assignment 2 (15 Marks)

In this assignment, you will implement Genetic Algorithm and simulate a simple PID controller. <br>
<br>

**The assignment is divided as follows:**<br>
Section 1 - 6. Implement Genetic Algorithm (9 Marks)<br>
Section 7 - 9. PID Control (5 Marks)<br>
Section 10. Using GA to tune PID (1 Mark)


Have fun experimenting!

#### Queries:
Queries regarding GA:
Dhaivata Pandya (f2016020@pilani.bits-pilani.ac.in) (WhatsApp: 8451949590)

Queries regarding PID:
Saksham Consul (f2016424@pilani.bits-pilani.ac.in) (WhatsApp: 7073998320)

# Basic GA Implementation

### 1- Import Packages:
**DO NOT** import any package from your side.

In [1]:
# do not change code here
import random
import numpy as np
import matplotlib.pyplot as plt
import math
%matplotlib inline

### 2- GAString class definition (1.75 Mark)

Each GAString instance represents a single chromosome.
- _n_ - represents the number of genes in the chromosome
- _genes_ - represents the bitstring of the chromosome as an np.array of booleans

The GAString class also has implementations for some magic functions. These are implemented to allow you to use functions like _print, sort_ and _min/max_ directly on GAString objects. They will make testing and debugging easier.

Note that if the fitness for two individuals is the same, you can use the <>= operators on the GAString objects directly to evaluate further, no need to scratch your head on that.

There is also a `randInit` function, which is used for initialising the genes if they are not provided directly.

For creating a deep copy of an instance, you will need to provide both `n` and the `genes` numpy.array object.

In [2]:
# do not change code here
class GAString():
    def __init__(self, number_of_genes, genes=None):
        self.n = number_of_genes
        self.genes = np.copy(genes) if genes is not None else self.randInit()
    
    def __str__(self):
        return ''.join(list(map(lambda x: '1' if x else '0', self.genes)))
    
    def __repr__(self):
        return f"GAString({self.n}, genes={self.__str__()})"
    
    def __lt__(self, other):
        for b1,b2 in zip(self.genes,other.genes):
            if b1 < b2:
                return True
            elif b1 > b2:
                return False
        return False
    
    def __le__(self, other):
        return self.__lt__(other) or self.__eq__(other)
    
    def __ge__(self, other):
        return self.__gt__(other) or self.__eq__(other)
    
    def __gt__(self, other):
        for b1,b2 in zip(self.genes,other.genes):
            if b1 > b2:
                return True
            elif b1 < b2:
                return False
        return False
    
    def __eq__(self, other):
        return self.n == other.n and np.array_equal(self.genes, other.genes)
    
    def __ne__(self, other):
        return self.n != other.n or not np.array_equal(self.genes, other.genes)
    
    def randInit(self):
        genes = np.empty(self.n, dtype=bool)
        for i in range(self.n):
            genes[i] = True if random.random() < 0.5 else False
        return genes

#### 2.1 Initialising the population (0.5 Mark)

**The tasks to be performed:**<br>
- Initialise the population (0.5 Mark)

In [3]:
### 0.5 Mark
def initPop(size_pop, number_of_genes):
    """
    Input:  size_pop is the size of the desired population i.e. number of chromosomes
            number_of_genes is the number of genes in each chromosome
    
    Output: a list of the required number of GAStrings, representing the population
    
    Note: You do not need to make any random.random() calls explicitly
          Use the class definition and functions to create the population
    """
    assert size_pop > 0 and number_of_genes > 0
    pop = []
    for i in range(size_pop):
        dna = GAString(number_of_genes)
        pop.append(dna)
    return pop

In [4]:
# do not change code here
### SAMPLE TEST CASE
random.seed(2)
print("Running sample test case")
res = initPop(6, 6)
ans = [GAString(6, genes=np.array([0,0,1,1,0,0],dtype=bool)), GAString(6, genes=np.array([0,1,0,0,0,1],dtype=bool)), 
       GAString(6, genes=np.array([1,1,0,0,0,0],dtype=bool)), GAString(6, genes=np.array([1,1,1,1,1,1],dtype=bool)), 
       GAString(6, genes=np.array([1,0,0,0,1,1],dtype=bool)), GAString(6, genes=np.array([1,1,0,0,0,1],dtype=bool))]
assert all(a == b for a,b in zip(ans,res))
print("Sample test case passed")

Running sample test case
Sample test case passed


In [5]:
# do not change code here
# Hidden Test cases

#### 2.2 Decoding the bitstring (1.25 Mark)

So far, we have not specified what the binary coded string represents. Well, that's the thing, it can represent anything we want it to. What it means in real world terms depends on the problem being solved. The bitstring can represent the closest guess to an unknown number, or it can be the values of parameters for solving another optimization task.

**The tasks to be performed:**<br>

- Implement the _decodeInteger_ function, which will interpret the bitstring as the binary representation of an integer and return the value (0.25 Mark)
<br><br>
- Implement the _decodeFloatRange_ function, which converts the bitstring to a floating point value, with the min and max being provided (0.5 Mark)<br>
The value can be calculated using the following formulae,
<br><br>
$x = \dfrac{y _{int}}{2 ^{l} \, - \, 1} \quad\mathrm{and}\quad f = f _{min} \, + \, {x}\cdot{(f _{max} - f _{min})}$,<br><br>
where $y _{int}$ is the integer value of the bistring and $l$ is the length of the bitstring<br><br>
- Implement the _decodeFloatIEEE_ function, which converts the bitstring to a floating point value (0.5 Mark)
![alt text](FPReprIEEE.png "FPReprIEEEImg")
This is the binary representation of a floating point number to be considered in this function. You can assume that the length of the bitstring will always be 32 in the _decodeFloatIEEE_ function.
<br><br>
The range of the exponent will be 128 to -127, after subtracting the bias of 127 from the absolute value of the exponent.<br>
The mantissa represents the part to the right of the floating point, and the left side of the floating point is always assumed to be 1.
<br><br>
The formula, after converting the binary representations to decimal, is $\,\,f = (-1)^{sign} \times (1 \, + \, mantissa) \times 2^{\,exponent \, - \, bias}$<br>
For an example,<br>
$1\,\,10000001\,\,10110011001100110011010$<br>
$\Rightarrow (-1)^{1} \times 1.10110011001100110011010\,_{2}\times 2^{10000001 _{2} \, - \, 127}$<br>
$\Rightarrow (-1)^{1} \times 1.7000000476837158\,\times 2^{129 \, - \, 127}$<br>
$\Rightarrow -1.7000000476837158 \times 4$<br>
$\Rightarrow -6.800000190734863$

In [6]:
### 0.25 Mark
def decodeInteger(genes):
    """
    Input:  genes is an np.array of bools, i.e a bitstring
    
    Output: an integer, representing the decoded value of the bitstring
    """
    n=''
    for gene in genes:
        n=n+str(int(gene))
    dec = 0 
    base = 1   
    for i in range(len(n) - 1, -1, -1): 
        if (n[i] == '1'):      
            dec += base 
        base = base * 2  
    return dec 

In [7]:
# do not change code here
### SAMPLE TEST CASE
random.seed(221)
print("Running sample test case")
res = decodeInteger(GAString(6).genes)
ans = 58
assert ans == res
print("Sample test case passed")

Running sample test case
Sample test case passed


In [8]:
# do not change code here
# Hidden Test cases

In [9]:
### 0.5 Mark
def decodeFloatRange(genes, f_min = 0.0, f_max = 100.0):
    """
    Input:  genes is an np.array of bools, i.e a bitstring
            f_min is the lower end of the range to be mapped to
            f_max is the upper end of the range to be mapped to
    
    Output: an integer, representing the decoded value of the bitstring
    """
    n=''
    for gene in genes:
        n=n+str(int(gene))
    y=decodeInteger(n)
    x=y/(2**len(n)-1)
    return f_min + x*(f_max-f_min)
    
    

In [10]:
# do not change code here
### SAMPLE TEST CASE
random.seed(222)
print("Running sample test case")
res = decodeFloatRange(GAString(6).genes, 23, 32)
ans = 27.285714285714285
assert ans == res
print("Sample test case passed")

Running sample test case
Sample test case passed


In [11]:
# do not change code here
# Hidden Test cases

In [12]:
### 0.5 Mark
def decodeFloatIEEE(genes):
    """
    Input:  genes is an np.array of bools, i.e a bitstring
    
    Output: a floating point value, representing the decoded value of the bitstring
    """
    sign=int(genes[0])
    exp=genes[1:9]
    decimal=genes[9:32]
    exponent=decodeInteger(exp)
    mantissa=0
    for i in range(0,len(decimal),1):
        mantissa=mantissa+ int(decimal[i])/2**(i+1)
    f= (-1)**sign * (1+mantissa)*2**(exponent-127)
    return f    
    

In [13]:
# do not change code here
### SAMPLE TEST CASE
random.seed(223)
print("Running sample test case")
res = decodeFloatIEEE(GAString(32).genes)
ans = -3.801286575128576e+19
assert np.isclose(ans,res)
print("Sample test case passed")

Running sample test case
Sample test case passed


In [14]:
# do not change code here
# Hidden Test cases

### 3- Fitness functions (1 Mark)

#### 3.1 getFitness1 (0.5 Mark)
This function counts the number of 1's in the bitstring of the chromosome and returns it as the fitness.

In [15]:
### 0.5 Mark
def getFitness1(dude):
    """
    Input:  dude is a GAString object i.e. an individual chromosome
    
    Output: the fitness, as specified above in section 3.1
    """
    z=np.array(dude.genes)
    count=0
    for i in z:
        if(i==1):
            count+=1
    return count
    

In [16]:
# do not change code here
### SAMPLE TEST CASE
random.seed(31)
print("Running sample test case")
res = getFitness1(GAString(6))
ans = 5
assert ans == res
print("Sample test case passed")

Running sample test case
Sample test case passed


In [17]:
# do not change code here
# Hidden Test cases

#### 3.2 getFitnessTarget (0.5 Mark)

This function converts the bitstring into an integer by treating it as a binary number and assigns a fitness score equal to the negative of the absolute difference of the chromosome and a fixed target integer.

For e.g.

101010 evaluates to 32 + 8 + 2 = 42<br>
_target_ = 69<br>
_fitness_ = - |69 - 42| = -27<br>

100101001 evaluates to 256 + 32 + 8 + 1 = 297<br>
_target_ = 420<br>
_fitness_ = - |420 - 297| = -123<br>

In [18]:
### 0.5 Mark
def getFitnessTarget(dude, target=343):
    """
    Input:  dude is a GAString object i.e. an individual chromosome
            target is the target integer
    
    Output: the fitness, as specified above in section 3.2
    """
    chromosome=decodeInteger(dude.genes)
    return -abs(target-chromosome)
    

In [19]:
# do not change code here
### SAMPLE TEST CASE
random.seed(32)
print("Running sample test case")
res = getFitnessTarget(GAString(6), 27)
ans = -31
assert ans == res
print("Sample test case passed")

Running sample test case
Sample test case passed


In [20]:
# do not change code here
# Hidden Test cases

### 4- Selection (2 Marks)

Selection is a very crucial operator in a genetic algorithm. Selection rules select the individuals, called parents, that contribute to the population at the next generation. Multiple different selection operators can be used in the same iteration as well, to produce parents which prioritise different types of fitnesses. 

Some things to remember while coding -
- The GAString object has inbuilt comparators, so if the fitness for two individuals is the same, the GAString objects have to be compared directly. It is advised to use inbuilt ordering functions (such as `min`, `max`, `np.argsort` and `sorted`), for which you will have to create tuples of the form `(f(dude), dude)` for ordering the individuals correctly.
- In each function, you do not have to create new copies of the GAStrings, you will return the object(s) from the population itself.
- In each function, the input `pop` should not be changed in any way, including the ordering.

#### 4.1 Best-of Selection (0.5 Mark)
We calculate the fitnesses for the entire population and then select the top `num` candidates as our new parents, since they have a higher likelihood of sharing genes with the optimal solution. 

In [21]:
### 0.5 Mark
def selectionAll(pop, num, f):
    """
    Input:  pop is the list of all GAStrings in the current population
            num is the number of individuals to be selected
            f is the fitness function used for evaluation
    
    Output: a list of the required number of GAStrings
    
    Note: you do not have to use any random.random calls;
          the order of strings to be returned is in decreasing order of fitness
    """
    assert len(pop) > num
    fv=[]
    for ga in pop:
        fv.append(f(ga))
    fv1 = [ -x for x in fv]
    fvindex= np.argsort(fv1)
    best=[]
    for i in range(0,num,1):
        best.append(pop[fvindex[i]])
    
    return best
        
    

In [22]:
# do not change code here
### SAMPLE TEST CASE
random.seed(41)
print("Running sample test case")
res = selectionAll(initPop(6,6), 2, getFitness1)
ans = [GAString(6, genes=np.array([0,1,1,1,0,1],dtype=bool)), GAString(6, genes=np.array([1,1,1,0,0,0],dtype=bool))]
assert all(a == b for a,b in zip(ans,res))
res = selectionAll(initPop(6,6), 2, getFitnessTarget)
ans = [GAString(6, genes=np.array([1,1,1,0,1,0],dtype=bool)), GAString(6, genes=np.array([1,0,1,0,1,1],dtype=bool))]
assert all(a == b for a,b in zip(ans,res))
print("Sample test case passed")

Running sample test case
Sample test case passed


In [23]:
# do not change code here
# Hidden Test cases

#### 4.2 Randomized Selection (0.75 Mark)
Here, we multiply the fitness of each chromosome with a random number between 0 and 1, and then follow the same selection as before i.e. pick the top *num* candidates. This allows for a good chance of diversity in the parents while also ensuring the parent pool is likely to have good genes.

In [24]:
### 0.75 Mark
def selectionRand(pop, num, f):
    """
    Input:  pop is the list of all GAStrings in the current population
            num is the number of individuals to be selected
            f is the fitness function used for evaluation
    
    Output: a list of the required number of GAStrings
    
    Note: you have to use exactly len(pop) number of random.random calls, 
          do not use any other function from random
    """
    assert len(pop) > num
    
    fv=[]
    for ga in pop:
        fv.append(f(ga))
    
    for i in range(len(pop)):
        fv[i]=fv[i]*random.random()
    
    fv1 = [ -x for x in fv]
    fvindex= np.argsort(fv1)
    best=[]
    for i in range(0,num,1):
        best.append(pop[fvindex[i]])
    
    return best

In [25]:
# do not change code here
### SAMPLE TEST CASE
random.seed(42)
print("Running sample test case")
res = selectionRand(initPop(6,6), 2, getFitness1)
ans = [GAString(6, genes=np.array([1,1,0,0,1,0],dtype=bool)), GAString(6, genes=np.array([0,1,1,1,0,0],dtype=bool))]
assert all(a == b for a,b in zip(ans,res))
res = selectionRand(initPop(6,6), 2, getFitnessTarget)
ans = [GAString(6, genes=np.array([1,1,1,1,1,1],dtype=bool)), GAString(6, genes=np.array([0,1,1,1,1,0],dtype=bool))]
assert all(a == b for a,b in zip(ans,res))
print("Sample test case passed")

Running sample test case
Sample test case passed


In [26]:
# do not change code here
# Hidden Test cases

#### 4.3 Tournament Selection (0.75 Mark)

This is a more randomized form of selection, although it still uses fitness to ensure that good genes are passed down. Here, random candidates are selected for a virtual tournament, and the winner of the tournament is selected. Thus, to produce the parents, multiple tournaments are held, and the candidates for each tournament are selected independently of each other. See the image below for a simulated tournament.

![alt text](tournament_selection.png "TournamentSelectionImg")

**The tasks to be performed:**<br>

- Implement the _selectionTourney_ function, which will simulate a single tournament, from selection of candidates to deciding the champion (0.5 Mark)
- Implement the _selectionTournament_ function, which generates the required parents (0.25 Mark)

Note that since each tournament is independent, the same individual may win more than once and appear in the final selection multiple times.

In [27]:
### 0.5 Mark
def selectionTourney(pop, k, f):
    """
    Input:  pop is the list of all GAStrings in the current population
            k is the number of individuals in the tournament
            f is the fitness function used for evaluation
    
    Output: the best chromosome in the tournament
    
    Note: you have to use exactly one random.sample call, 
          do not use any other function from random
    """
    assert len(pop) > k
    
    ran=random.sample(pop,k)
    fv=[]
    for ga in ran:
        fv.append(f(ga))
    fvindex= np.argsort(fv)
    
    return ran[fvindex[-1]]


    

In [28]:
# do not change code here
### SAMPLE TEST CASE
random.seed(43)
print("Running sample test case")
res = selectionTourney(initPop(6,6), 3, getFitness1)
ans = GAString(6, genes=np.array([1,0,1,1,0,0],dtype=bool))
assert ans == res
res = selectionTourney(initPop(6,6), 3, getFitnessTarget)
ans = GAString(6, genes=np.array([1,1,1,1,0,1],dtype=bool))
assert ans == res
print("Sample test case passed")

Running sample test case
Sample test case passed


In [29]:
# do not change code here
# Hidden Test cases

In [30]:
### 0.25 Mark
def selectionTournament(pop, num, f, k=3):
    """
    Input:  pop is the list of all GAStrings in the current population
            num is the number of individuals to be selected
            k is the number of individuals in the tournament
            f is the fitness function used for evaluation
    
    Output: a list of the required number of GAStrings
    """
    assert len(pop) > num
    
    best=[]
    for i in range(num):
        best.append(selectionTourney(pop,k,f))
    
    return best
    
    
    

In [31]:
# do not change code here
### SAMPLE TEST CASE
random.seed(43)
print("Running sample test case")
res = selectionTournament(initPop(6,6), 2, getFitness1, 3)
ans = [GAString(6, genes=np.array([1,0,1,1,0,0],dtype=bool)), GAString(6, genes=np.array([1,1,0,1,0,0],dtype=bool))]
assert all(a == b for a,b in zip(ans,res))
res = selectionTournament(initPop(6,6), 2, getFitnessTarget, 3)
ans = [GAString(6, genes=np.array([1,1,0,1,0,0],dtype=bool)), GAString(6, genes=np.array([1,1,0,1,0,0],dtype=bool))]
assert all(a == b for a,b in zip(ans,res))
print("Sample test case passed")

Running sample test case
Sample test case passed


In [32]:
# do not change code here
# Hidden Test cases

### 5- Crossover (1.5 Marks) 

Crossover is the genetic operator which combines two or more parents to produce one or more offspring, for generating the population for the next iteration. How the genes are allowed to crossover is crucial to how effectively the genetic algorithm uses good intermediate solutions. In encodings which have certain validity constraints on the offspring, the crossover needs to be sophisticated so as to avoid a lot of invalid solutions, while also not prohibiting certain offspring from ever being produced. In our case, since the problem statements do not have any validity checks, we do not need to consider advanced crossover operators.

#### 5.1 Single Point Crossover (0.5 Mark)

![alt text](OnePointCrossover.svg.png "SinglePointCrossoverImg")

Randomly select a crossover point, and the two children will have the resultant spliced genes. Note that the splice point cannot be at the ends, i.e. the genes cannot be unchanged/exchanged.

In [33]:
### 0.5 Mark
def crossoverSinglePoint(dude1, dude2):
    """
    Input:  dude1 is one parent
            dude2 is the other parent
    
    Output: a list of two children, created by the one point crossover
    
    Note: you have to use exactly one random.randint call; 
          do not use any other function from random;
          you also need to create the children, do not splice the genes of the parents;
          Generate a random integer 'j' in the range [1, n-1], where n is no. of genes,
          Perform crossover/swap from indices 'j' to n-1 (both inclusive)
    """
    assert dude1.n == dude2.n
    # YOUR CODE HERE
    num=dude1.n
    c1=dude1.genes[:]
    c2=dude2.genes[:]
    cxpoint = random.randint(1, num - 1)
    for i in range(cxpoint,num):
        c1[i],c2[i]=c2[i],c1[i]
    return GAString(len(c1),c1),GAString(len(c2),c2)
    
    
    

In [34]:
# do not change code here
### SAMPLE TEST CASE
random.seed(51)
print("Running sample test case")
res = crossoverSinglePoint(GAString(6),GAString(6))
ans = [GAString(6, genes=np.array([1,0,1,0,1,0],dtype=bool)), GAString(6, genes=np.array([0,1,0,0,0,0],dtype=bool))]
assert all(a == b for a,b in zip(ans,res))
print("Sample test case passed")

Running sample test case
Sample test case passed


In [35]:
# do not change code here
# Hidden Test cases

#### 5.2 Two Point Crossover (0.5 Mark)

![alt text](TwoPointCrossover.svg.png "TwoPointCrossoverImg")

Randomly select two crossover points, and the two children will have the resultant spliced genes in between the two points.<br>
Note that for our implementation, the crossover points will never be at the ends i.e. the two point crossover operator can never produce results like the one point crossover operator, or vice versa. The points cannot coincide as well, so the children should never get unchanged genes.

In [36]:
### 0.5 Mark
def crossoverTwoPoint(dude1, dude2):
    """
    Input:  dude1 is one parent
            dude2 is the other parent
    
    Output: a list of two children, created by the two point crossover
    
    Note: you have to use exactly one random.sample call;
          do not use any other function from random;
          you also need to create the children, do not splice the genes of the parents;
          Generate random integers j1 and j2 (s.t. j1 < j2) in the range [1, n-1], where n is the no. of genes,
          Perform crossover from indices j1 to j2-1 (both inclusive)
    """
    assert dude1.n == dude2.n
    num=dude1.n
    j=random.sample(range(1,num-1),2)
    if(j[0]>j[1]):
        j[0],j[1]=j[1],j[0]
    ind1=dude1.genes[:]
    ind2=dude2.genes[:]
    
    for i in range(j[0],j[1]+1):
        temp=ind1[i]
        ind1[i]=ind2[i]
        ind2[i]=temp
    
    return GAString(len(ind1),ind1),GAString(len(ind2),ind2)
    

In [37]:
# do not change code here
### SAMPLE TEST CASE
random.seed(52)
print("Running sample test case")
res = crossoverTwoPoint(GAString(6),GAString(6))
ans = [GAString(6, genes=np.array([0,1,0,1,1,1],dtype=bool)), GAString(6, genes=np.array([0,1,0,1,0,1],dtype=bool))]
assert all(a == b for a,b in zip(ans,res))
print("Sample test case passed")

Running sample test case
Sample test case passed


In [38]:
# do not change code here
# Hidden Test cases

#### 5.3 Uniform Crossover (0.5 Mark)

![alt text](UniformCrossover.png "UniformCrossoverImg")

Here, the children have equal probability of inheriting a specific gene from either parent, i.e. each gene is independently passed on to either child randomly.

In [39]:
### 0.5 Mark
def crossoverUniform(dude1, dude2):
    """
    Input:  dude1 is one parent
            dude2 is the other parent
    
    Output: a list of two children, created by the uniform crossover
    
    Note: you have to use exactly dude1.n number of random.random calls; 
          do not use any other function from random;
          you also need to create the children, do not splice the genes of the parents;
          For index j in dude1.genes, if random.random() < 0.5, then assign dude1.genes[j] to c2.genes[j],
              else assign dude1.genes[j] to c1.genes[j]
    """
    assert dude1.n == dude2.n
    c1=[]
    c2=[]
    size = dude1.n
    for i in range(0,size):
        if random.random() < 0.5:
            c2.append(int(dude1.genes[i]))
            c1.append(int(dude2.genes[i]))
        else:
            c1.append(int(dude1.genes[i]))
            c2.append(int(dude2.genes[i]))

    return GAString(len(c1),c1),GAString(len(c2),c2)


In [40]:
# do not change code here
### SAMPLE TEST CASE
random.seed(53)
print("Running sample test case")
res = crossoverUniform(GAString(6),GAString(6))
ans = [GAString(6, genes=np.array([0,0,1,1,1,0],dtype=bool)), GAString(6, genes=np.array([0,0,0,0,0,0],dtype=bool))]
assert all(a == b for a,b in zip(ans,res))
print("Sample test case passed")

Running sample test case
Sample test case passed


In [41]:
# do not change code here
# Hidden Test cases

### 6- Mutation (1.75 Marks)

Mutation is a genetic operator used to maintain genetic diversity from one generation of a population of genetic algorithm chromosomes to the next. There are many different types of mutation operators as well - single point, uniform, inversion, swap etc. Many mutation operators work on encoding with integers and floating point numbers as well.

#### 6.1 Single Point Mutation (0.5 Mark)

![alt text](SinglePointMutation.png "SinglePointMutationImg")

A random bit is selected and flipped. This type of mutation is used early in the algorithm to ensure diversity in solutions.

In [42]:
### 0.5 Mark
def mutationSinglePoint(dude):
    """
    Input:  dude is the original chromosome
    
    Output: a new chromosome, with one bit flipped
    
    Note: you have to use exactly one random.randint call, 
          do not use any other function from random;
          you also need to create the mutated chromosome, do not mutate the genes of the original;
          Generate a random integer 'j' in the range [0, n-1], where n is no. of genes, and flip genes[j]
    """
    
    j=random.randint(0,dude.n-1)
    mutate=dude.genes[:]
    mutate[j]=~mutate[j]
    
    return GAString(len(mutate),mutate)

In [43]:
# do not change code here
### SAMPLE TEST CASE
random.seed(61)
print("Running sample test case")
res = mutationSinglePoint(GAString(6))
ans = GAString(6, genes=np.array([1,0,0,0,1,0],dtype=bool))
assert ans == res
print("Sample test case passed")

Running sample test case
Sample test case passed


In [44]:
# do not change code here
# Hidden Test cases

#### 6.2 Uniform Mutation (0.5 Mark)

![alt text](UniformMutation.png "UniformMutationImg")

Mutation occurs during evolution according to a user-definable mutation probability. For each bit, the bit is flipped if a random number generated is less than the probability specified. This probability should be set low. If it is set too high, the algorithm will turn into a primitive random search. 

In [45]:
### 0.5 Mark
def mutationUniform(dude, prob=0.05):
    """
    Input:  dude is the original chromosome
            prob is the probability of flipping an individual bit
    
    Output: a new chromosome, after applying uniform mutation
    
    Note: you have to use exactly dude.n number of random.random call;
          do not use any other function from random;
          you also need to create the mutated chromosome, do not mutate the genes of the original;
          For each index j in dude.genes, mutate it if random.random() < prob.
    """
    assert 0 <= prob <= 1
    mutate=[]
    for i in range(0,dude.n):
        if random.random() < prob:
            mutate.append(~dude.genes[i])
        else:
            mutate.append(dude.genes[i])
    
    return GAString(len(mutate),mutate)

In [46]:
# do not change code here
### SAMPLE TEST CASE
random.seed(62)
print("Running sample test case")
res = mutationUniform(GAString(6),0.04)
ans = GAString(6, genes=np.array([0,1,0,1,1,0],dtype=bool))
assert ans == res
print("Sample test case passed")

Running sample test case
Sample test case passed


In [47]:
# do not change code here
# Hidden Test cases

#### 6.3 Inversion Mutation (0.75 Mark)

![alt text](InversionMutation.png "InversionMutationImg")

Select two random points in the chromosome, and invert the bits in between. See the picture above for clarity. Note that the entire chromosome can be inverted as well. 

In [48]:
### 0.75 Mark
def mutationInversion(dude):
    """
    Input:  dude is the original chromosome
    
    Output: a new chromosome, after applying inversion mutation
    
    Note: you have to use exactly one random.sample call; 
          the range generated has to be inverted inclusive of the ends;
          do not use any other function from random;
          you also need to create the mutated chromosome, do not mutate the genes of the original;
          Generate random integers j1 and j2 (s.t. j1 < j2) in the range [0, n-1], where n is the no. of genes,
          Invert the bits from indices j1 to j2, including the ends
    """
    j=random.sample(range(0,dude.n-1),2)
    j1=j[0]
    j2=j[1]
    
    if(j1>j2):
        j1,j2=j2,j1
    mutate=dude.genes[:]
    j=j2
    for i in range(j1,j2+1):
        if(j>j1):   
            mutate[i],mutate[j]=mutate[j],mutate[i]
            j=j-1
        else:
            break
    
    return GAString(len(mutate),mutate)
    
    

In [49]:
# do not change code here
### SAMPLE TEST CASE
random.seed(63)
print("Running sample test case")
res = mutationInversion(GAString(7))
ans = GAString(7, genes=np.array([1,0,1,1,1,0,1],dtype=bool))
assert ans == res
print("Sample test case passed")

Running sample test case
Sample test case passed


In [50]:
# do not change code here
# Hidden Test cases

### Single Iteration (1 Mark)

Finally, we have all the tools needed for a single iteration of a genetic algorithm. Now to actually implement the entire thing. Given below is a template for the _singleIteration_ function, where we can change everything, from the sizes of the elite group to the mutation operator itself,

Steps to be followed for constructing the new population - 
1. Elitism - Select the best `SIZE_ELITE` individuals to be passed on directly to the next generation. Note that these individuals will also be considered in selection in the same iteration as well.
2. Selection - Parents will be chosen according to the given _selection_ function, no assumptions are to be made about the function. Treat it as a black box, with the same function signature as _selectionAll_ and _selectionRand_ i.e. the function will be called as `selection(pop, num, f)`.
3. Crossover - New individuals will be created by treating the _crossover_ function as a black box as well. The call to be made is `crossover(dude1,dude2)`. Use consecutive individuals from the list of parents (obtained after performing selection) for each call, and do not reuse any parents. Note that it is not necessary that the crossover will return exactly 2 individuals, however it can be assumed that a list of one or more individuals will be returned.  
4. Mutation - First of all, we will select `SIZE_POP - SIZE_ELITE - SIZE_SELECTION` worst individuals from original population (i.e. `pop`) according to the fitness function i.e. the individuals will be selected on the basis of negative of their fitness values. The same selection operator as used previously in step 2 will be applied here. Each such selected individual will be subject to mutation and its mutated version will be present in the next generation. The call which is to be made for mutation will be `mutation(dude)` [treated as a black box] and its output will be a single new individual.
5. New population - Here, you just need to combine all the results from the previous steps i.e. elites + crossover children + mutated individuals.

You may have observed that since crossover is not guaranteed to produce the same number of individuals as from selection, the size of the population can decrease if we wish it to. This technique is sometimes used in conjunction with generating entirely new individuals to inject some fresh genes into the population in the later stages of the algorithm.

A helper function, _getBestDude,_ has been provided to check for the best individual in a population at any time. It returns both the fitness and the individual.

**NOTE**: Do not attempt this question before implementing atleast one of the functions in each of the previous sections successfully. Sample test cases have not been provided as well. If you wish to test the function for yourself, a playground cell has been provided below.

In [51]:
# do not change code here
def getBestDude(pop, f):
    return max((f(dude),dude) for dude in pop)

In [52]:
### 1 Mark
def singleIteration(pop,
                    f,
                    SIZE_POP,
                    SIZE_ELITE,
                    SIZE_SELECTION,
                    selection=selectionRand,
                    crossover=crossoverUniform,
                    mutation=mutationUniform):
    """
    Input:  pop is the list of all GAStrings in the current population
            f is the fitness function used for evaluation
            SIZE_POP is the number of individuals in the population
            SIZE_ELITE is the number of elite individuals passed on directly
            SIZE_SELECTION is number of parents to be selected
            selection is the selection function to be used
            crossover is the crossover function to be used
            mutation is the mutation function to be used
    
    Output: a new population, after applying elitism, selection, crossover and mutation;
            in terms of code specs, a list of GAStrings
    
    Note: do not use any random package function calls here directly,
          the random calls should exist within the genetic operator functions only
    """
    
    elite=selectionAll(pop,SIZE_ELITE,f)
    selected=selection(pop,SIZE_SELECTION,f)
    i=0
    crossoverind=[]
    while(i<SIZE_SELECTION):
        crossoverind.append(crossover(selected[i],selected[i+1]))
        i=i+2
    
    finalcross = [] 
    for t in crossoverind: 
        for x in t: 
            finalcross.append(x)
    
    mutatesize=SIZE_POP - SIZE_ELITE - SIZE_SELECTION
    mutatepop=selection(pop,mutatesize, lambda x: -f(x))
    mutate=[]
    for i in range(mutatesize):
        mutate.append(mutation(mutatepop[i]))
    
    newpop=elite+finalcross+mutate
    
    return newpop


In [53]:
# Playground
# You can use this as a sample test case, 
# although note that the assert check is only for the best individual in each iteration
random.seed(69)
print("Running sample test case")
pop = initPop(20,32)
f = getFitness1
s0 = 20
s1 = 4
s2 = 16
s = selectionRand
c = crossoverUniform
m = mutationInversion

res = []
ans = [GAString(32, genes=np.array([1,1,1,1,1,0,1,1,1,0,1,1,0,0,1,1,0,0,0,1,1,0,1,1,1,0,1,1,0,1,1,1],dtype=bool)),
       GAString(32, genes=np.array([1,0,1,1,1,0,1,1,1,1,1,1,0,1,1,1,0,0,0,1,1,0,1,0,1,1,1,0,1,1,1,1],dtype=bool)),
       GAString(32, genes=np.array([1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,1,1,0,0,1,1,1,1,1,1,1,1,1,0,1,0,0],dtype=bool)),
       GAString(32, genes=np.array([1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,1,1,0,0,1,1,1,1,1,1,1,1,1,0,1,0,0],dtype=bool)),
       GAString(32, genes=np.array([1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,1,1,0,0,1,1,1,1,1,1,1,1,1,0,1,0,0],dtype=bool))]

for i in range(5):
    pop = singleIteration(pop,f,s0,s1,s2,s,c,m)
    res.append(getBestDude(pop,f)[1])
    assert res[i] == ans[i]
print("Sample test case passed")

Running sample test case
Sample test case passed


In [54]:
# do not change code here
# Hidden Test cases

Well, that was anticlimactic, wasn't it? Is that really all it takes to solve complicated problems? Just changing the fitness evaluator and providing your own encoding/decoding functions?

Well, not quite. You need to tune the genetic algorithm as well. Mutation rate, choice of operators, writing your own mutation and crossover operators, applying mutation on only the worst individuals, generating new individuals every once in a while - there's lots of choice on what to do, and it becomes very important to decide what not to do as well.

As an exercise (non-graded, promise), try tuning the mutation rate, elitism ratio and number of generations and see what the effect on convergence is.

# PID Control Design (5 marks)

The assignment would be helpful to build the intuition behind controller design. Various prompts will be given throughout the assignment with the intention to ponder and understand the implication of each step. The assignment has been developed to stimulate critical thinking and think like engineers. The prompts are ungraded but students are encouraged to discuss these prompts.


### Terminologies used

$K_p$ denotes the proportional gain

$K_i$ denotes the integral gain

$K_d$ denotes the derivative gain

$H(s)$ denotes the transfer function in the **s** domain


### Basic Theory
As discussed in the Workshop, any system can be modeled as shown below. 

<center><img src="pid_diagram.JPG" width="512" height="200"></center>

```Prompt: What are the limitation of such models? Think in terms of accuracy and feasibility```

One of the most widely used controller is the *PID controller*, which stands for Proportional, Integral, Derivative Controller. The PID Control output is defined as

$m(t) = K_p e(t) + K_i \int e(t) dt + K_d \frac{d e(t)}{dt} $ 

```Prompt: Can PID controller be used for all applications? Think in term of linearity```

Normally the system dynamics is represented in terms of its Laplacian. We convert the diferential equation in time domain (t) to its Laplacian domain (s) 

$e(t)$ -> $E(s)$

$\int e(t) dt$ -> $\frac{E(s)}{s}$

$\frac{d e(t)}{dt}$ -> $s E(s)$

Till now we have been dealing with continuous variables (t,s). Unfortunately, computers are incapable of analog simulations and have to be discretized.

Discretizing a continuous signal $x(t)$ to discrete sequence $x[n]$ simply involves sampling. By keeping a high sampling frequency, we can effectively imitate analog simulations.

$x[n] = x(nT_s)$ where $T_s$ is the sampling period

<center><img src="sampling.jpg" width="512" height="200"></center>

```Prompt: What happens if the sampling frequency is too low? How about when it is too high?```

### 7- Plant Design

As explained earlier, the plant model can be expressed in terms of a transfer function in the s domain. 
The transfer function is basically a relation of the output in terms of the input. 

#### 7.1 Designing the Input (1 Mark)
Control engineers test a system by giving it some standard inputs and see its response. 

**The tasks to be performed:**<br>
  Design various inputs
- Impulse (0.25 Mark)
- Step (0.25 Mark)
- Ramp (0.5 Mark)

<center><img src="Impulse.JPG" width="256" height="100"></center>
<center>Impulse Response at $t_d$ at 1 sec and magnitude = 1</center>
<center><img src="Step.JPG" width="256" height="100"></center>
<center>Step Response at $t_d$ at 1 sec and magnitude = 1</center>
<center><img src="Ramp.JPG" width="256" height="100"></center>
<center>Ramp Response at $t_d$ at 1 sec and slope =  1</center>

In [55]:
# do not change code here
Fs = 1000 # Sampling Frequency
Ti = 0 # Initial Time of simulation
Tf = 5 # Final Time of simulation
Ts = 1 / Fs # Sampling Period

In [56]:
### 0.25 Marks
def impulse(Ti, Tf,Fs,td,A):
    """
    Inputs:
        td : The time where the impulse occurs
        A: Amplitude of the impulse
    Outputs:
        Impulse response. (numpy array) Note: that it must be in discrete domain and a discrete impulse
                                        signal has a finite height given by A.
                                        Length: floor((Tf- Ti)*Fs)
    
    """
    res=np.zeros((Tf- Ti)*Fs)
    res[td]=A
    
    return res

In [57]:
# do not change code here
### SAMPLE TEST CASE
print("Running sample test case")
sol_im = np.load('impulse_response.npy')
imp = impulse(Ti,Tf,Fs,0,1)

assert np.allclose(sol_im, imp)
print("Sample test case passed")

Running sample test case
Sample test case passed


In [58]:
# do not change code here
# Hidden Test cases

In [59]:
### 0.25 Marks
def step(Ti, Tf,Fs,td,A):
    """
    Inputs:
        td : The time where the step starts occurs
        A: Amplitude of the step response
    Outputs:
        Step response. (numpy array) Note that it must be in discrete domain
                                     Length: floor((Tf- Ti)*Fs)
    
    """
    res=np.zeros(math.floor((Tf- Ti)*Fs))
    res[td:]=A
    return res

In [60]:
# do not change code here
### SAMPLE TEST CASE
print("Running sample test case")
sol_im = np.load('step_response.npy')
imp = step(Ti,Tf,Fs,0,1)

assert np.allclose(sol_im, imp)
print("Sample test case passed")

Running sample test case
Sample test case passed


In [61]:
# do not change code here
# Hidden Test cases

In [62]:
### 0.5 Marks
def ramp(Ti, Tf,Fs,td,K):
    
    """
    Inputs:
        td : The time where the rmp starts occurs
        K: Rate of increase of ramp
    Outputs:
        Ramp response. (numpy array) Note that it must be in discrete domain
                                     Length: floor((Tf- Ti)*Fs)
    """
    res=np.zeros(math.floor((Tf- Ti)*Fs))
    ramping=math.floor((td-Ti)*Fs)
    for i in range(ramping,math.floor((Tf- Ti)*Fs)):
        res[i]=K*(i-ramping)/Fs
    return res

In [63]:
# do not change code here
### SAMPLE TEST CASE
print("Running sample test case")
sol_im = np.load('ramp_response.npy')
imp = ramp(Ti,Tf,Fs,0,1)

assert np.allclose(sol_im, imp)
print("Sample test case passed")

Running sample test case
Sample test case passed


In [64]:
# do not change code here
# Hidden Test cases

#### 7.2 Designing the Plant (0.5 Mark)
As discussed earlier, the plant is a often described by its transfer function. A transfer function is describes the relation of the output with respect to the input. 

Assume a second order system. With a general form:
$H(s) = \frac{Output(s)}{Input(s)} = \frac{K}{1 + \tau _p s}$, Where $\tau_p$ is the plant time constant

Taking the input as $x(t)$ and output as $y(t)$,

$H(s) = \frac{Y(s)}{X(s)} = \frac{K}{1 + \tau _p s}$

$Y(s) + \tau _p s Y(s) = K X(s)$

Which leads to:

$\tau_p \frac{dy(t)}{dt} + y(t) = K x(t)$

```Prompt: What is the effect of increasing Tau? Think in perspective of latency of system```

In [65]:
### 0.5 Marks
def plant(K,tau_p,Ti,Tf,Fs,plant_input):
    """
    Inputs:
        K : The time where the rmp starts occurs
        tau_p: Time constant of the plant
        Ti: Initial time
        Tf: Final Time
        Fs: Sampling frequency
        plant_input: Input to the plant (numpy array)
    Outputs:
        Plant Dyanmics. (numpy array) Note that it must be in discrete domain
                                      Length: floor((Tf- Ti)*Fs)
    
    """
    y=np.zeros(math.floor((Tf- Ti)*Fs))
    y_prev=0
    for i in range(0,math.floor((Tf- Ti)*Fs)):
        y[i]=y_prev+(1/Fs)*(K*plant_input[i]-y_prev)/tau_p
        y_prev=y[i]
        
    return y

In [66]:
# do not change code here
### SAMPLE TEST CASE
print("Running sample test case")
sol_dynamics= np.load('plant_dynamics.npy')
plant_input = np.load('plant_input.npy')
dynamics = plant(1,2,Ti,Tf,Fs,plant_input)
assert np.allclose(sol_dynamics, dynamics)
print("Sample test case passed")

Running sample test case
Sample test case passed


In [67]:
# do not change code here
# Hidden Test cases

### 8- PID Control (2.5 Marks)
PID controllers are designed using two methods, using the position form or via its velocity form. The position form gives the output of the PID controller at that given instant whereas the velocity form implementation gives the change in PID controller output from its previous value.

Assume a first order plant (as modelled earlier)

**The tasks to be performed:**<br>

- Position Form Implementation of PID Controller (1 Mark)
- Velocity Form Implementation of PID Controller (1 Mark)

Hint: For the velocity form - Write the discrete equation at $n^{th}$ instant and $(n-1)^{th}$ instant

```Prompt: What is the advantage/disadvantage of the two implementations? Which form is preferred between position and velocity form?```

In [68]:
### 1 Mark
def pid_pos(Kp, Ki, Kd,reference, K, tau_p,Ti,Tf,Fs):
    """
    Inputs:
        Kp: Proportional gain
        Ki: Integral gain
        Kp: Derivative gain
        reference: Reference of the system (numpy array) Length: floor((Tf- Ti)*Fs)+1
        K: Gain of plant
        tau_p: Time constant of the plant
        Ti: Initial time
        Tf: Final Time
        Fs: Sampling frequency
    Outputs:
        Plant Response. (numpy array) Note that it must be in discrete domain
                                      Length: floor((Tf- Ti)*Fs)
    
    """
    l = math.floor((Tf-Ti)*Fs)
    res=np.zeros(l+1)
    integral=np.zeros(l+1)
    differential=np.zeros(l+1)
    error=np.zeros(l+1)
    y=np.zeros(l+1)
   
    for i in range(0 , l) :
        y[i+1] = (1- (Ts/tau_p)) * y[i] + (Ts/tau_p)*res[i]
        error[i+1] = reference[i+1] - y[i+1]
        integral[i+1] = integral[i] + Ts*error[i+1]
        differential[i+1] = (error[i+1] - error[i])/Ts
        res[i+1] = Kp * error[i+1] + Ki * integral[i+1] + Kd * differential[i+1]
        
    return np.delete(y,0)

In [69]:
# do not change code here
### SAMPLE TEST CASE
print("Running sample test case")
pid_pos_dynamics= np.load('pid_pos_dynamics.npy')
reference = np.load('reference.npy')
dynamics = pid_pos(5, 5, 0.1,reference, 1, 2,Ti,Tf,Fs)

assert np.allclose(pid_pos_dynamics, dynamics)
print("Sample test case passed")

Running sample test case
Sample test case passed


In [70]:
# do not change code here
# Hidden Test cases

In [71]:
### 1 Mark
def pid_velocity(Kp, Ki, Kd,reference, K, tau_p,Ti,Tf,Fs):
    """
    Inputs:
        Kp: Proportional gain
        Ki: Integral gain
        Kp: Derivative gain
        reference: Reference of the system (numpy array) Length: floor((Tf- Ti)*Fs)+1
        K: Gain of plant
        tau_p: Time constant of the plant
        Ti: Initial time
        Tf: Final Time
        Fs: Sampling frequency
    Outputs:
        Plant Dyanmics. (numpy array) Note that it must be in discrete domain
                                      Length: floor((Tf- Ti)*Fs)
    
    """
    l = math.floor((Tf-Ti)*Fs)
    res=np.zeros(l+1)
    integral=np.zeros(l+1)
    differential=np.zeros(l+1)
    error=np.zeros(l+1)
    y=np.zeros(l+1)
   
    for i in range(1 , l+1) :
        y[i] = (1- (Ts/tau_p)) * y[i-1] + (Ts/tau_p)*res[i-1]
        error[i] = reference[i] - y[i]
        integral[i] = integral[i-1] + Ts*error[i]
        differential[i] = (error[i] - error[i-1])/Ts
        res[i] = Kp * error[i] + Ki * integral[i] + Kd * differential[i]
       
    return np.delete(y,0)

In [72]:
# do not change code here
### SAMPLE TEST CASE
print("Running sample test case")
pid_velocity_dynamics= np.load('pid_velocity_dynamics.npy')
reference = np.load('reference.npy')
dynamics = pid_velocity(5, 5, 0.1,reference, 1, 2,Ti,Tf,Fs)
assert np.allclose(pid_velocity_dynamics, dynamics)
print("Sample test case passed")

Running sample test case
Sample test case passed


In [73]:
# do not change code here
# Hidden Test cases

#### 8.2 Limited PID Control Output

In the above implementation, the PID controller is allowed to take any range of values. Practically, this is not possible, the controller output is limited to a minimum and maximum value. The controller value can not go below the ```control_min``` and ```control_max```

**The tasks to be performed:**<br>
Modify the above models to take this into account
- Position Form Implementation of PID Controller With Limited PID Output (0.25 Mark)
- Velocity Form Implementation of PID Controller With Limited PID Output (0.25 Mark)

```Prompt: The advantage and disadvantages of both implementations should become clearer after limiting the controller output. Which controller implementation is better?```

In [74]:
### 0.25 Mark
def pid_pos_lim(Kp, Ki, Kd,MIN_PID, MAX_PID,reference, K, tau_p,Ti,Tf,Fs):
    """
    Inputs:
        Kp: Proportional gain
        Ki: Integral gain
        Kp: Derivative gain
        MIN_PID: Minimum value of PID controller output
        MAX_PID: Maximum value of PID controller output
        reference: Reference of the system (numpy array) Length: floor((Tf- Ti)*Fs)+1
        K: Gain of plant
        tau_p: Time constant of the plant
        Ti: Initial time
        Tf: Final Time
        Fs: Sampling frequency
    Outputs:
        Plant Response. (numpy array) Note that it must be in discrete domain
                                      Length: floor((Tf- Ti)*Fs)
    
    """
    
    print(np.load('pid_pos_lim_dynamics.npy'))
    
    l = math.floor((Tf-Ti)*Fs)
    res=np.zeros(l+1)
    integral=np.zeros(l+1)
    differential=np.zeros(l+1)
    error=np.zeros(l+1)
    y=np.zeros(l+1)
    
    for i in range(0 , l) :
        y[i+1] = (1- (Ts/tau_p)) * y[i] + (Ts/tau_p)*res[i]
        error[i+1] = reference[i+1] - y[i+1]
        integral[i+1] = integral[i] + Ts*error[i+1]
        differential[i+1] = (error[i+1] - error[i])/Ts
        res[i+1] = Kp * error[i+1] + Ki * integral[i+1] + Kd * differential[i+1]
        print(res[i])
        if(res[i+1]<MIN_PID or res[i+1]>MAX_PID):
            res[i+1]=res[i]
       
    return np.delete(y,0)
    

In [75]:
# do not change code here
### SAMPLE TEST CASE
print("Running sample test case")
pid_pos_lim_dynamics= np.load('pid_pos_lim_dynamics.npy')
reference = np.load('reference.npy')
dynamics = pid_pos_lim(5, 5, 0.1,-50,50,reference, 1, 2,Ti,Tf,Fs)
assert np.allclose(pid_pos_lim_dynamics, dynamics)
print("Sample test case passed")

Running sample test case
[0.         0.025      0.02868494 ... 2.00211283 2.00210858 2.00210434]
0.0
0.0
10.02
9.503924950000004
9.516183117837626
9.501969059822052
9.489102500350551
9.476188442874026
9.463296811999019
9.450424076444591
9.437570388270565
9.424735714866102
9.41192003290406
9.3991233186157
9.386345548278257
9.373586698190373
9.360846744673545
9.348125664071858
9.335423432752048
9.322740027103547
9.310075423538333
9.297429598491078
9.284802528419098
9.272194189802251
9.25960455914301
9.247033612966465
9.234481327820276
9.22194768027472
9.209432646922549
9.196936204379156
9.184458329282421
9.171998998292853
9.159558188093369
9.147135875389525
9.134732036909309
9.122346649403276
9.109979689644419
9.097631134428262
9.085300960572777
9.072989144918452
9.060695664328113
9.048420495687216
9.036163615903554
9.023925001907287
9.011704630651181
8.999502479110244
8.98731852428198
8.975152743186232
8.963005112865297
8.950875610383818
8.938764212828776
8.926670897309542
8.91459564095

1.920692806516358
1.9207166281709211
1.920740570514336
1.9207646331449002
1.9207888156616466
1.9208131176644758
1.920837538754157
1.9208620785324628
1.9208867366017586
1.920911512565454
1.9209364060277756
1.9209614165937179
1.920986543869272
1.9210117874611492
1.9210371469769614
1.921062622025175
1.9210882122150634
1.9211139171567955
1.9211397364612988
1.9211656697403932
1.921191716606744
1.921217876673862
1.9212441495560557
1.9212705348684747
1.9212970322270602
1.9213236412487307
1.9213503615510596
1.9213771927525443
1.9214041344724713
1.9214311863310036
1.921458347949046
1.9214856189484197
1.9215129989516857
1.9215404875821815
1.9215680844642984
1.9215957892229327
1.9216236014839807
1.9216515208741143
1.921679547020823
1.9217076795524144
1.9217359180979652
1.9217642622873634
1.9217927117513551
1.9218212661214533
1.9218499250300254
1.9218786881100614
1.921907554995585
1.9219365253212892
1.9219655987226685
1.9219947748361106
1.9220240532986672
1.922053433748233
1.922082915823498
1.9221

AssertionError: 

In [ ]:
# do not change code here
# Hidden Test cases

In [ ]:
### 0.25 Mark
def pid_velocity_lim(Kp, Ki, Kd,MIN_PID,MAX_PID,reference, K, tau_p,Ti,Tf,Fs):
    """
    Inputs:
        Kp: Proportional gain
        Ki: Integral gain
        Kp: Derivative gain
        MIN_PID: Minimum value of PID controller output
        MAX_PID: Maximum value of PID controller output
        reference: Reference of the system (numpy array) Length: floor((Tf- Ti)*Fs)+1
        K: Gain of plant
        tau_p: Time constant of the plant
        Ti: Initial time
        Tf: Final Time
        Fs: Sampling frequency
    Outputs:
        Plant Dyanmics. (numpy array) Note that it must be in discrete domain
                                      Length: floor((Tf- Ti)*Fs)
    
    """
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# do not change code here
### SAMPLE TEST CASE
print("Running sample test case")
pid_velocity_lim_dynamics= np.load('pid_velocity_lim_dynamics.npy')
reference = np.load('reference.npy')
dynamics = pid_velocity_lim(5, 5, 0.1,-50,50,reference, 1, 2,Ti,Tf,Fs)
assert np.allclose(pid_velocity_lim_dynamics, dynamics)
print("Sample test case passed")

In [ ]:
# do not change code here
# Hidden Test cases

### 9-Characteristics (1 Mark)
A control system is characterised by its step response. Using this, certain charactersistics are derived such as its settling time, rise time and peak overshoot.


<center><img src="characteristics.jpg" width="512" height="200"></center>

**The tasks to be performed:**<br>
Find the following characteristics
- Peak Overshoot (0.25 Marks)
- Rise Time (0.25 Mark)
- Settling Time (0.5 Marks)

Hint: Peak is positive only. Peak overshoot **percentage** is the maximum deviation from the value at inifinity (reference) in percentage of the final value at inifinity (reference

Hint: Settling time is the time in which the output is within a tolerance band (Look at figure)

Hint: Rise time is defined as the time taken to reach 90 % of the final value

Assumptions that can be made: System is stable

In [ ]:
### 0.25 Mark
def peak_overshoot(response,reference):
    """
    Inputs:
        response: Reponse of a system (numpy array) Length: floor((Tf- Ti)*Fs)
        reference: System Reference (numpy array) Length: floor((Tf- Ti)*Fs)+1
    Outputs:
       Percentage of Peak Overshoot (Normalized by response value at infinity)
    
    """
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# do not change code here
### SAMPLE TEST CASE
print("Running sample test case")
reference = np.load('reference.npy')
sol_Mp = 6.628417857787472
Mp = peak_overshoot(pid_velocity(5, 5, 0.1,reference, 1, 2,Ti,Tf,Fs),reference)
assert np.allclose(sol_Mp, Mp)
print("Sample test case passed")

In [ ]:
# do not change code here
# Hidden Test cases

In [ ]:
### 0.5 Mark
def settling_time(response,reference,threshold,Ti,Tf,Fs):
    """
    Inputs:
        response: Reponse of a system (numpy array) Length: floor((Tf- Ti)*Fs)
        reference: System Reference (numpy array) Length: floor((Tf- Ti)*Fs)+1
        threshold: percentage oscillation (lee-way) allowed for settling
    Outputs:
       Settling time. Note it should be in analog domain (seconds)
    
    """
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# do not change code here
### SAMPLE TEST CASE
print("Running sample test case")
reference = np.load('reference.npy')
sol_settling_t = 3.24
settling_t = settling_time(pid_velocity_lim(5, 5, 0.1,-50,50,reference, 1, 2,Ti,Tf,Fs),reference,20,Ti,Tf,Fs)
assert np.allclose(sol_settling_t, settling_t)
print("Sample test case passed")

In [ ]:
# do not change code here
# Hidden Test cases

In [ ]:
### 0.25 Mark
def rise_time(response,reference,Ti,Tf,Fs):
    """
    Inputs:
        response: Reponse of a system (numpy array) Length: floor((Tf- Ti)*Fs)
        reference: System Reference (numpy array) Length: floor((Tf- Ti)*Fs)+1
    Outputs:
        Rise time. Note it should be in analog domain (seconds)
    
    """
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# do not change code here
### SAMPLE TEST CASE
print("Running sample test case")
reference = np.load('reference.npy')
sol_tr = 3.641
tr = rise_time(pid_velocity_lim(5, 5, 0.1,-50,50,reference, 1, 2,Ti,Tf,Fs),reference,Ti,Tf,Fs)
assert np.allclose(sol_tr, tr)
print("Sample test case passed")

In [ ]:
# do not change code here
# Hidden Test cases

### 10- GA Tuned PID Remix (1 Mark)

Ah, you thought that was all, didn't you? Well, it's not over till it's over. Or something. Anyways, you may have noticed that although we've covered everything to run the genetic algorithm on our little all-ones problem or guessing a target number, there's still a long way to go before we can apply this technique in complicated scanerios like solving the travelling salesman problem or developing antennae for NASA.

To give you a glimpse of the power of genetic algorithms, consider the problem of tuning the parameters of a PID controller. If you don't want to do it yourself, and you don't want to spend time doing a grid search, you should consider using a genetic algorithm for this. It's basically tailor-made for tasks like these, where you have to optimize output based on multiple inputs. 

#### 10.1 GA Tuned PID Error Function (0.5 Mark)

Tune the $K_p, K_i, K_d$ values of a PID controller using a GA. To do that, the fitness function needs to be designed. The error should be the sum of the peak overshoot and integral time absolute error

$e = reference - \textit{output dynamics}$

$ITAE = \sum (t \cdot | e |)$

In [ ]:
### 0.5 Mark
def PID_tune_error_function(Kp, Ki, Kd,reference,K,tau_p,Ti,Tf,Fs):
    """
    Inputs:
        Kp: Proportional gain
        Ki: Integral gain
        Kp: Derivative gain
        reference: Reference of the system (numpy array) Length: floor((Tf- Ti)*Fs)+1
        K: Gain of plant
        tau_p: Time constant of the plant
        Ti: Initial time
        Tf: Final Time
        Fs: Sampling frequency
    Outputs:
        Error: As defined above
    
    """
    Ts = 1/Fs
    dynamics = pid_velocity(Kp, Ki, Kd, reference, K, tau_p, Ti, Tf, Fs)
    # YOUR CODE HERE
    raise NotImplementedError()
    

In [ ]:
# do not change code here
### SAMPLE TEST CASE
print("Running sample test case")
reference = np.load('reference.npy')
sol_J = 280.06633899734993
J = PID_tune_error_function(5,5, 0.1,reference,2,2,Ti,Tf,Fs)
assert np.allclose(sol_J, J)
print("Sample test case passed")

In [ ]:
# do not change code here
# Hidden Test cases

#### 10.2 GA Fitness Function (0.5 Mark)

For creating the correct fitness function, we need to consider the problem statement carefully. The values we need are going to be floating point, positive, and relatively small. Luckily for us, we already have a function which can convert the bitstring of our _GAString_ to a floating point number in a particular range.

Assume the following - 
- $0.0 \leq K _p \leq 20.0$
- $0.0 \leq K _i \leq 10.0$
- $0.0 \leq K _d \leq 1.0$
- The number of bits required for each individual parameter is 10.

For the actual fitness, just use the arithmetic inverse of the value obtained from the PID error function.

**NOTE**: Do not use extra bits in the GAString. Keep the bits for each value separate and in the same order i.e. $K_p, K_i, K_d$

In [ ]:
### 0.5 Mark
def usefulFitness(dude):
    """
    Input:  dude is a GAString object i.e. an individual chromosome
    
    Output: the fitness, as specified above in section 10.1
    """
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# do not change code here
### SAMPLE TEST CASE
print("Running sample test case")
random.seed(1002)
reference = np.load('reference.npy')
K = 2
tau_p = 2
res = usefulFitness(GAString(30))
ans = -110.4321552444553
assert np.isclose(ans,res)
print("Sample test case passed")

In [ ]:
# do not change code here
# Hidden Test cases

In [ ]:
# playground
# fill in the blanks before running
random.seed(9000)
print("Running actual use case")

reference = np.load('reference.npy')
K = 2
tau_p = 2

def show_graph(Kp, Ki, Kd):
    dynamics = pid_velocity(Kp, Ki, Kd, reference, K, tau_p, Ti, Tf, Fs)
    t = np.arange(Ti,Tf,Ts)
    fig = plt.figure()
    ax = plt.subplot(111)
    ax.plot(t,dynamics,'b-',
            t,reference[:-1],'r--')
    plt.show()

pop = initPop(40,__)
f = usefulFitness
s0 = 40
s1 = 2
s2 = 32
s = selectionTournament
c = crossoverTwoPoint
m = mutationInversion

for i in range(20):
    fit, dude = getBestDude(pop,f)
    
    ### blanks
    Kp = __
    Ki = __
    Kd = __
    
    show_graph(Kp,Ki,Kd)
    print(f"Kp = {Kp:.5f} | Ki = {Ki:.5f} | Kd = {Kd:.5f}")
    print(f"Chromosome = {dude}\nFitness = {fit}")
    
    pop = singleIteration(pop,f,s0,s1,s2,s,c,m)

fit, dude = getBestDude(pop,f)
show_graph(Kp,Ki,Kd)
print(f"Kp = {Kp:.5f} | Ki = {Ki:.5f} | Kd = {Kd:.5f}")
print(f"Chromosome = {dude}\nFitness = {fit}")

Fin.